In [1]:
import pandas as pd 

In [11]:
from datetime import date

In [16]:
envios = pd.read_csv('../data/raw/envios.txt', sep='|')

In [17]:
ventas = pd.read_csv('../data/raw/ventas.txt', sep='|')

In [18]:
ventas.rename(columns={'fecha':'fecha_venta'}, inplace=True)

In [19]:
ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'])

In [20]:
ventas['fecha_venta_norm'] = ventas['fecha_venta'].apply(lambda x : date(x.year,x.month,1))

In [24]:
ventas = ventas.groupby(['id_pos','fecha_venta_norm','canal']).unidades.sum().reset_index()

In [26]:
pos = pd.read_csv('../data/raw/pos.csv')

In [27]:
pos = pos[pos['id_pos'].isnull() != True]

In [28]:
pos = pos[pos['id_pos']!='Not Available']

In [30]:
ventas.shape

(35818, 4)

In [29]:
# convertimos a int para poder cruzar con informacion de venta
pos['id_pos'] = pos['id_pos'].astype(int)

In [31]:
train = pd.merge(ventas, pos, how='inner', on='id_pos')

In [32]:
train.shape

(34540, 36)

In [36]:
train.to_csv('../data/raw/train_aggr.csv', sep=';', index=False)

In [37]:
print("Num. id_pos de ventas: ",ventas.id_pos.nunique())

Num. id_pos de ventas:  12302


In [38]:
print("Num. id_pos de train (ventas x pos) : ",train.id_pos.nunique())

Num. id_pos de train (ventas x pos) :  11853


In [39]:
print("Num. id_pos no encontrados: " , ventas.id_pos.nunique() - train.id_pos.nunique())

Num. id_pos no encontrados:  449


### Terminar: Falta agregar lo de envios que hay que tener en cuenta la info de las fechas, fecha_venta > fecha_envio

In [45]:
# Nos quedamos con un unico id_pos, sin tener en cuenta la fecha
envios_tmp = envios[['id_pos']].drop_duplicates()

In [52]:
train[train.id_pos.isin(envios.id_pos)].id_pos.nunique()

12300

In [51]:
train[train.id_pos.isin(envios.id_pos)!=True].id_pos.nunique()

2

In [46]:
train = pd.merge(train, envios_tmp, how='left', left_on='id_pos', right_on='id_pos')

In [47]:
train.shape

(266436, 36)

In [48]:
train.head()

,fecha,id_pos,unidades,canal,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,...,pct_bachelors,pct_doctorados,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar
0,2012-10-01,8495,4,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.260,...,16.750,0.610,32.42,5.35,0.180,1.160,0.00,0.145,3.01,152962.485
1,2012-10-10,8571,1,MAXIKIOSCO,10.0,100524.20,134309.315,5403.320,4.250,5.765,...,25.215,3.675,16.68,13.85,0.075,2.915,3.18,0.005,0.87,325699.190
2,2012-10-11,8915,1,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
3,2012-10-12,8915,3,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
4,2012-10-13,8923,1,MAXIKIOSCO,9.0,31019.49,43710.370,4117.760,5.240,6.120,...,6.610,0.240,36.29,4.82,0.230,3.250,0.00,0.000,2.20,81577.770


In [26]:
submittion.id_pos.nunique()

12896

In [29]:
submittion.dtypes

id_pos    int64
score     int64
dtype: object

In [37]:
submittion[submittion.id_pos.isin(pos.id_pos)]['id_pos'].nunique()

12896